# INSPIRER_1.0

Fetch a random image from Google Drive

In [161]:
import random
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth() # client_secrets.json need to be in the same directory as the script
drive = GoogleDrive(gauth)

In [ ]:
file_list = drive.ListFile({'q': "'{Google drive folder ID}' in parents and trashed=false"}).GetList()
todays_img = random.choice(file_list)
print('title: %s, id: %s' % (todays_img['title'],todays_img['id']))
todays_img.GetContentFile(todays_img['title'])

Email the image as an attachment

In [163]:
import smtplib
import ssl
from email.mime.text import MIMEText
from email.utils import formataddr
from email.mime.multipart import MIMEMultipart  # New line
from email.mime.base import MIMEBase  # New line
from email import encoders  # New line
import os

In [ ]:
# User configuration
sender_email = "{sender@email.com}"
sender_name = "{Mr. Sender}"
password = "{email password}" #app Password from GMail

receiver_emails = ["{receiver@email.com}"]
receiver_names = ["Mr. Receiver"]

# Email body
email_html = open('email.html')
email_body = email_html.read()

filename = todays_img['title']

for receiver_email, receiver_name in zip(receiver_emails, receiver_names):
		print("Sending the email...")
		# Configurating user's info
		msg = MIMEMultipart()
		msg['To'] = formataddr((receiver_name, receiver_email))
		msg['From'] = formataddr((sender_name, sender_email))
		msg['Subject'] = "Here's some inspiration for you!"
		
		msg.attach(MIMEText(email_body, 'html'))

		try:
			# Open PDF file in binary mode
			with open(filename, "rb") as attachment:
							part = MIMEBase("application", "octet-stream")
							part.set_payload(attachment.read())

			# Encode file in ASCII characters to send by email
			encoders.encode_base64(part)

			# Add header as key/value pair to attachment part
			part.add_header(
					"Content-Disposition",
					f"attachment; filename= {filename}",
			)

			msg.attach(part)
		except Exception as e:
				print(f'Oh no! We didn\'t find the attachment!\n{e}')
				break

		try:
				# Creating a SMTP session | use 587 with TLS, 465 SSL and 25
				server = smtplib.SMTP('smtp.gmail.com', 587)
				# Encrypts the email
				context = ssl.create_default_context()
				server.starttls(context=context)
				# We log in into our Google account
				server.login(sender_email, password)
				# Sending email from sender, to receiver with the email body
				server.sendmail(sender_email, receiver_email, msg.as_string())
				print('Email sent!')
		except Exception as e:
				print(f'Oh no! Something bad happened!\n{e}')
				break
		finally:
				print('Closing the server...')
				server.quit()

Remove the file from the local directory to save space

In [165]:
os.remove(todays_img['title'])